In [1]:
%matplotlib inline
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
from tools.common import disp_grid
from IPython.display import Image
from skimage.io import imsave

from keras.models import model_from_json
import os
import joblib
import numpy as np
from lightjob.cli import load_db
#from skimage.io import imread
from tools.common import find_generation_job, find_training_job, to_generation, compute_sample_objectness
from IPython.display import Image, display
from lightjob.cli import load_db
from lightjob.db import SUCCESS
import pandas as pd

/home/mcherti/work/code/external/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
db = load_db()
jobs = db.jobs_with(state=SUCCESS, where='jobset75')
S = [j['summary'] for j in jobs]
stats = {s: pd.read_csv('../jobs/results/{}/csv/stats.csv'.format(s)) for s in S}

In [3]:
jobs_gen = to_generation(jobs)

In [23]:
#model_folder = '../../nnbench/out/feature_generation/5_vs_fake_jobset75'
model_folder = '../tools/models/mnist/m2'
model = model_from_json(open(os.path.join(model_folder, 'model.json')).read())
model.load_weights(os.path.join(model_folder, 'model.h5'))

## checking all generated

In [5]:
d = np.load('../exported_data/figs/5_vs_fake_jobset75.npz')
y = d['y']
data = d['X']
#data = data[71536:] # test data
#y = y[71536:] # test data
data = data[y==10]/255.
print(data.shape)

(84000, 1, 28, 28)


In [ ]:
print(set(data.flatten()))

In [ ]:
preds = model.predict(data)

In [ ]:
scores = compute_sample_objectness(preds)
print(scores)
v = np.array(preds)

marginal = v.mean(axis=0)
score = v * np.log(v)
score = score.sum(axis=1)
score = np.exp(score)
# nan scores are scores where v.min() == v.max() (generated only zero images)

[ 6.61398125  6.80477333  3.54022813 ...,  2.59081054  4.19062614
  3.08430243]


In [ ]:
CLASS = 5
p=preds[:, CLASS]
img = data[np.argsort(p)[::-1]]
#img = img[ (p[np.argsort(p)[::-1]])>0.5 ]
img = disp_grid(img[0:121], border=1, bordercolor=(0.3, 0, 0))
imsave('imgs/out.png', img)
Image('imgs/out.png')

## checking stats of jobs

In [ ]:
db = load_db()
jobs = db.jobs_with(state=SUCCESS, where='jobset75')
S = [j['summary'] for j in jobs]

In [ ]:
jobs_gen = to_generation(jobs, db=db)

In [ ]:
field = 'stats.out_of_the_box_classification.m2.objectness'
indices = np.arange(len(jobs))
for i in range(len(jobs_gen)):
    j = jobs_gen[i]
    v = db.get_value(j, field, if_not_found=None)
    if not v:print('Misssing generation jobs from training job : {}, skip it.'.format(jobs[i]['summary']))

In [ ]:
import shutil
field = 'stats.out_of_the_box_classification.m2.objectness'
indices = np.arange(len(jobs))
objectness = map(lambda j:db.get_value(j, field, if_not_found=np.nan), jobs_gen)
objectness = np.array(objectness)
indices = filter(lambda ind:not np.isnan(objectness[ind]), indices)
indices = sorted(indices, key=lambda i:objectness[i])
indices = indices[::-1]
images = ['../exported_data/figs/generated/jobset{:05d}/{}.png'.format(int(jobs[i]['where'][6:]), jobs_gen[i]['summary']) 
          for i in indices]
print(jobs_gen[indices[1]]['summary'])
for i in range(len(images)):
    print(jobs_gen[indices[i]]['summary'])
    shutil.copy(images[i], 'export/{:05d}.png'.format(i))
    #display(Image(images[i]), width=100, height=100)